In [1]:
# Libraries
from sklearn.preprocessing import OneHotEncoder
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import os
import pandas as pd
from scipy.stats import zscore
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from sklearn.metrics import mean_squared_error
user = os.getenv('USERPROFILE')
data_path = os.path.join(
    user, 'OneDrive - National University of Singapore\EBAC\Year 1 Semester 1\Project\Data')
os.chdir(data_path)
pd.set_option('max_columns', None)

In [31]:
season_df = pd.read_excel('Events Calendar.xlsx',
                          sheet_name='POC2', engine='openpyxl')

season_df.loc[season_df['Phoon Huat Season'] == 'Y', 'Phoon Huat Season'] = 1
season_df = season_df.dropna()
season_df = season_df.drop_duplicates(subset=['DATE'])

In [11]:
retail_df = pd.read_csv('Retail_Week_NUS.csv', parse_dates=[0])
retail_df.head()

,DATE,CUSTNAME,MATERIAL,BASEUOM,MSTAE,H1,H2,H3,QTY_SOLD,CLUSTER
0,2018-06-25,Store 4,10091,PAC,AC,Dairy,Cream Compound,Whipping,5.0,B
1,2018-06-25,Store 4,10550,PAC,AC,"Nuts, Seeds & Beans",Nuts,Pumpkin Kernel,3.0,B
2,2018-06-25,Store 4,11485,PAC,AC,"Flour, Grain & Flakes",Premix,Others,1.0,B
3,2018-06-25,Store 4,1201,PAC,AC,Bakery,Functional,Starches,1.0,B
4,2018-06-25,Store 4,12085,PAC,AC,Grocery,Seasoning,Herbs & Spices,1.0,B


In [12]:
retail_df['MATERIAL'].nunique()

10785

**Seasonal Items**

In [ ]:
seasonal_list = [116,    250,    328,    714,    738,    901,    909,    917,
                 972,    986,   1016,   1027,   1340,   1603,   2307,   2731,
                 7125,   8092,   8207,   8460,   8563,   9080,   9081,   9122,
                 9159,  10549,  10550,  10562,  10564,  10572,  10683,  10691,
                 10692,  10699,  10913,  11955,  13121,  13695,  13789,  14005,
                 14856,  14990,  14991,  20042,  51949,  52289,  52408,  52821,
                 52824,  52825,  52906,  53035,  53330,  53655,  53945,  53990,
                 53991,  54011,  54012,  54032,  54080,  54089,  54110,  55682,
                 56406,  57248,  57250,  57775,  57809,  58360,  58536,  58957,
                 59173,  59174,  61215,  61240,  61299,  61305,  61713,  62239,
                 62240,  63277,  65646,  66019,  66095,  66446,  67544,  67549,
                 67764,  67770,    799,   1605,   1606,   8091,   8093,   8122,
                 9545,   9890,  10033,  11656,  12481,  12501,  13120,  50071,
                 52069,  52889,  53005,  54016,  54022,  54029,  54538,  56276,
                 57060,  57813,  57814,  58256,  61715,  62065,  63840,  63841,
                 64382,  64622,  65642,  65879,  66027,  66040,  66503,  66947,
                 67049,  67248,  67551,    744,   1604,  10568,  10599,  14728,
                 15251,  64383,  51697,  67816,  13731,  13732,  67767,  68323,
                 100452,  52661]

# Data Cleaning

**We only look at active skus and remove General and Service in H1**

In [13]:
retail_df = retail_df.loc[(retail_df['MSTAE'] == 'AC') & (
    (retail_df['H1'] != 'General') & (retail_df['H1'] != 'Service'))].copy()

**Create Year and Week**

In [14]:
retail_df['YEAR'] = retail_df['DATE'].dt.year
retail_df['MONTH'] = retail_df['DATE'].dt.month
retail_df['WEEK'] = retail_df['DATE'].dt.strftime('%W')
print(retail_df.shape)
retail_df.head()

(1391820, 13)


,DATE,CUSTNAME,MATERIAL,BASEUOM,MSTAE,H1,H2,H3,QTY_SOLD,CLUSTER,YEAR,MONTH,WEEK
0,2018-06-25,Store 4,10091,PAC,AC,Dairy,Cream Compound,Whipping,5.0,B,2018,6,26
1,2018-06-25,Store 4,10550,PAC,AC,"Nuts, Seeds & Beans",Nuts,Pumpkin Kernel,3.0,B,2018,6,26
2,2018-06-25,Store 4,11485,PAC,AC,"Flour, Grain & Flakes",Premix,Others,1.0,B,2018,6,26
3,2018-06-25,Store 4,1201,PAC,AC,Bakery,Functional,Starches,1.0,B,2018,6,26
4,2018-06-25,Store 4,12085,PAC,AC,Grocery,Seasoning,Herbs & Spices,1.0,B,2018,6,26


**Insert weeks with no sales (Put 0)<br>Did not put currently as it affects standard deviation**

In [ ]:
# retail_group = retail_df.groupby(['YEAR', 'WEEK', 'MATERIAL'])[
#     'QTY_SOLD'].sum().reset_index()

In [ ]:
# retail_group.head()

In [ ]:
# retail_group = retail_df.pivot_table(index=['YEAR', 'WEEK'], columns=[
#     'MATERIAL'], values='QTY_SOLD', fill_value=0).reset_index()

In [ ]:
# retail_group = retail_group.melt(
#     id_vars=['YEAR', 'WEEK'], value_vars=retail_group.columns[2:], value_name='QTY_SOLD')

In [ ]:
# print(retail_group.shape)
# retail_group.head()

# Seasonal Products

**Sales of items affected by**
- Economy at times
- Nature of Item (Seasonal or Non-seasonal)
- Promotions or Discounts
- Competition from a rival
- Competition from a substitute product
- Special events like Super Bowl, Thanksgiving, New Year, etc varying from places to places

In [ ]:
avg_df = retail_group.groupby(['MATERIAL', 'YEAR'])[
    'QTY_SOLD'].mean().reset_index()
avg_df = avg_df.rename(columns={'QTY_SOLD': 'AVG_QTY'})
avg_df.head()

In [ ]:
retail_merged = retail_group.merge(avg_df, how='left', on=['YEAR', 'MATERIAL'])
retail_merged.head()

**Create seasonal index**

In [ ]:
retail_merged['SEASONAL_INDEX'] = retail_merged['QTY_SOLD'] / \
    retail_merged['AVG_QTY']

In [ ]:
std_df = retail_merged.groupby(['MATERIAL', 'YEAR'])[
    'SEASONAL_INDEX'].std().reset_index()
std_df = std_df.rename(columns={'SEASONAL_INDEX': 'STD'})

In [ ]:
retail_merged = retail_merged.merge(
    std_df, how='left', on=['MATERIAL', 'YEAR'])

**Remove materials that are not sold, it may be cause of lack of inventory**

In [ ]:
retail_merged = retail_merged.dropna(subset=['STD'])

retail_merged = retail_merged.groupby(['YEAR', 'MATERIAL'])['STD'].mean(
).to_frame().sort_values(by=['YEAR', 'STD'], ascending=False)

In [ ]:
high_std = retail_merged.loc[retail_merged['STD'] >= 2]
low_std = retail_merged.loc[retail_merged['STD'] < 1]

**Find common seasonality items for 2019 and 2020**

In [ ]:
s_products = high_std.loc[2019].merge(
    high_std.loc[2020], left_index=True, right_index=True)
non_s_products = low_std.loc[2019].merge(
    low_std.loc[2020], left_index=True, right_index=True)

In [ ]:
s_products

In [ ]:
def plot(mat, year):
    df = retail_df.loc[(retail_df['MATERIAL'] == mat)
                       & (retail_df['YEAR'] == year)]
    df.groupby(['DATE'])['QTY_SOLD'].sum().plot()

In [ ]:
plot(225, 2018)

In [ ]:
test = retail_df.loc[retail_df['MATERIAL'] == 52289]

In [ ]:
test.groupby(['DATE'])['QTY_SOLD'].sum().plot()

# Regression

In [15]:
retail_df['DATE'] = pd.to_datetime(retail_df['DATE']).dt.date

In [16]:
retail_df['DATE_LY'] = retail_df['DATE'] - relativedelta(years=1)
retail_df['DATE_LM'] = retail_df['DATE'] - relativedelta(months=1)
retail_df['DATE_LW'] = retail_df['DATE'] - relativedelta(weeks=1)

**Change to datetime**

In [17]:
retail_df['DATE_LY'] = retail_df['DATE_LY'].apply(pd.to_datetime)
retail_df['DATE_LM'] = retail_df['DATE_LM'].apply(pd.to_datetime)
retail_df['DATE_LW'] = retail_df['DATE_LW'].apply(pd.to_datetime)

In [18]:
retail_df['YEAR_LY'] = retail_df['DATE_LY'].dt.year
retail_df['MONTH_LY'] = retail_df['DATE_LY'].dt.month
retail_df['WEEK_LY'] = retail_df['DATE_LY'].dt.strftime('%W')

retail_df['YEAR_LM'] = retail_df['DATE_LM'].dt.year
retail_df['MONTH_LM'] = retail_df['DATE_LM'].dt.month
retail_df['WEEK_LM'] = retail_df['DATE_LM'].dt.strftime('%W')

retail_df['YEAR_LW'] = retail_df['DATE_LW'].dt.year
retail_df['MONTH_LW'] = retail_df['DATE_LW'].dt.month
retail_df['WEEK_LW'] = retail_df['DATE_LW'].dt.strftime('%W')

In [19]:
retail_df.head()

,DATE,CUSTNAME,MATERIAL,BASEUOM,MSTAE,H1,H2,H3,QTY_SOLD,CLUSTER,YEAR,MONTH,WEEK,DATE_LY,DATE_LM,DATE_LW,YEAR_LY,MONTH_LY,WEEK_LY,YEAR_LM,MONTH_LM,WEEK_LM,YEAR_LW,MONTH_LW,WEEK_LW
0,2018-06-25,Store 4,10091,PAC,AC,Dairy,Cream Compound,Whipping,5.0,B,2018,6,26,2017-06-25,2018-05-25,2018-06-18,2017,6,25,2018,5,21,2018,6,25
1,2018-06-25,Store 4,10550,PAC,AC,"Nuts, Seeds & Beans",Nuts,Pumpkin Kernel,3.0,B,2018,6,26,2017-06-25,2018-05-25,2018-06-18,2017,6,25,2018,5,21,2018,6,25
2,2018-06-25,Store 4,11485,PAC,AC,"Flour, Grain & Flakes",Premix,Others,1.0,B,2018,6,26,2017-06-25,2018-05-25,2018-06-18,2017,6,25,2018,5,21,2018,6,25
3,2018-06-25,Store 4,1201,PAC,AC,Bakery,Functional,Starches,1.0,B,2018,6,26,2017-06-25,2018-05-25,2018-06-18,2017,6,25,2018,5,21,2018,6,25
4,2018-06-25,Store 4,12085,PAC,AC,Grocery,Seasoning,Herbs & Spices,1.0,B,2018,6,26,2017-06-25,2018-05-25,2018-06-18,2017,6,25,2018,5,21,2018,6,25


In [20]:
# Get slice of dataframe first
retail_ly = retail_df[['YEAR', 'MONTH',
                       'WEEK', 'CUSTNAME', 'MATERIAL', 'QTY_SOLD']]
retail_lm = retail_df[['YEAR', 'MONTH',
                       'WEEK', 'CUSTNAME', 'MATERIAL', 'QTY_SOLD']]
retail_lw = retail_df[['YEAR', 'MONTH',
                       'WEEK', 'CUSTNAME', 'MATERIAL', 'QTY_SOLD']]

In [21]:
retail_created = retail_df[['DATE', 'CUSTNAME', 'MATERIAL',
                            'QTY_SOLD', 'YEAR_LY', 'MONTH_LY', 'WEEK_LY',
                            'YEAR_LM', 'MONTH_LM', 'WEEK_LM', 'YEAR_LW',
                            'MONTH_LW', 'WEEK_LW']]

**Get Retail Last Year**

In [22]:
retail_ly = retail_created.merge(retail_ly, how='left', left_on=['YEAR_LY', 'MONTH_LY', 'WEEK_LY',
                                                                 'CUSTNAME', 'MATERIAL'],
                                 right_on=['YEAR', 'MONTH', 'WEEK', 'CUSTNAME', 'MATERIAL'])
# retail_ly = retail_ly.fillna(0)
retail_ly = retail_ly.rename(
    columns={'QTY_SOLD_x': 'QTY_SOLD', 'QTY_SOLD_y': 'QTY_SOLD_LY'})
retail_ly = retail_ly.drop(columns=['YEAR', 'MONTH', 'WEEK'])

In [23]:
retail_ly_lm = retail_ly.merge(retail_lm, how='left', left_on=['YEAR_LM', 'MONTH_LM', 'WEEK_LM',
                                                               'CUSTNAME', 'MATERIAL'],
                               right_on=['YEAR', 'MONTH', 'WEEK', 'CUSTNAME', 'MATERIAL'])
# retail_ly = retail_ly.fillna(0)
retail_ly_lm = retail_ly_lm.rename(
    columns={'QTY_SOLD_x': 'QTY_SOLD', 'QTY_SOLD_y': 'QTY_SOLD_LM'})
retail_ly_lm = retail_ly_lm.drop(columns=['YEAR', 'MONTH', 'WEEK'])

In [37]:
retail_ly_lm_lw = retail_ly_lm.merge(retail_lw, how='left', left_on=['YEAR_LW', 'MONTH_LW', 'WEEK_LW',
                                                                     'CUSTNAME', 'MATERIAL'],
                                     right_on=['YEAR', 'MONTH', 'WEEK', 'CUSTNAME', 'MATERIAL'])
# retail_ly = retail_ly.fillna(0)
retail_ly_lm_lw = retail_ly_lm_lw.rename(
    columns={'QTY_SOLD_x': 'QTY_SOLD', 'QTY_SOLD_y': 'QTY_SOLD_LW'})
retail_ly_lm_lw = retail_ly_lm_lw.drop(columns=['YEAR', 'MONTH', 'WEEK'])
print(retail_ly_lm_lw.shape)
retail_ly_lm_lw.head()

(1391820, 16)


,DATE,CUSTNAME,MATERIAL,QTY_SOLD,YEAR_LY,MONTH_LY,WEEK_LY,YEAR_LM,MONTH_LM,WEEK_LM,YEAR_LW,MONTH_LW,WEEK_LW,QTY_SOLD_LY,QTY_SOLD_LM,QTY_SOLD_LW
0,2018-06-25,Store 4,10091,5.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN
1,2018-06-25,Store 4,10550,3.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN
2,2018-06-25,Store 4,11485,1.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN
3,2018-06-25,Store 4,1201,1.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN
4,2018-06-25,Store 4,12085,1.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN


In [38]:
season_df['DATE'] = season_df['DATE'].apply(pd.to_datetime)
retail_ly_lm_lw['DATE'] = retail_ly_lm_lw['DATE'].apply(pd.to_datetime)

In [39]:
retail_ly_lm_lw = retail_ly_lm_lw.merge(season_df, on='DATE', how='left')

In [43]:
retail_ly_lm_lw['Phoon Huat Season'] = retail_ly_lm_lw['Phoon Huat Season'].fillna(0)
print(retail_ly_lm_lw.shape)
retail_ly_lm_lw.head()

(1391820, 18)


,DATE,CUSTNAME,MATERIAL,QTY_SOLD,YEAR_LY,MONTH_LY,WEEK_LY,YEAR_LM,MONTH_LM,WEEK_LM,YEAR_LW,MONTH_LW,WEEK_LW,QTY_SOLD_LY,QTY_SOLD_LM,QTY_SOLD_LW,Holiday,Phoon Huat Season
0,2018-06-25,Store 4,10091,5.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN,NaN,0
1,2018-06-25,Store 4,10550,3.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN,NaN,0
2,2018-06-25,Store 4,11485,1.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN,NaN,0
3,2018-06-25,Store 4,1201,1.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN,NaN,0
4,2018-06-25,Store 4,12085,1.0,2017,6,25,2018,5,21,2018,6,25,NaN,NaN,NaN,NaN,0


In [47]:
retail_ly_lm_lw.to_csv('Retail_Features.csv',index=False)

In [48]:
df = retail_ly_lm_lw.copy()
df = df.dropna(subset=['QTY_SOLD_LY', 'QTY_SOLD_LM', 'QTY_SOLD_LW'])
df.head()

,DATE,CUSTNAME,MATERIAL,QTY_SOLD,YEAR_LY,MONTH_LY,WEEK_LY,YEAR_LM,MONTH_LM,WEEK_LM,YEAR_LW,MONTH_LW,WEEK_LW,QTY_SOLD_LY,QTY_SOLD_LM,QTY_SOLD_LW,Holiday,Phoon Huat Season
465336,2019-07-08,Store 4,1027,7.0,2018,7,27,2019,6,22,2019,7,26,3.0,6.0,5.0,NaN,0
465340,2019-07-08,Store 4,10386,3.0,2018,7,27,2019,6,22,2019,7,26,3.0,2.0,10.0,NaN,0
465341,2019-07-08,Store 4,10546,3.0,2018,7,27,2019,6,22,2019,7,26,5.0,8.0,3.0,NaN,0
465346,2019-07-08,Store 4,10681,2.0,2018,7,27,2019,6,22,2019,7,26,7.0,2.0,2.0,NaN,0
465348,2019-07-08,Store 4,10976,1.0,2018,7,27,2019,6,22,2019,7,26,2.0,3.0,2.0,NaN,0


In [51]:
df.to_csv('Retail_Features_Dropped.csv',index=False)

In [52]:
# Choose Date (Last Week)
date = df['DATE'].max() - relativedelta(days=7)
# Choose Date (70, 30)
# date = df.iloc[round(len(df) * 0.7), :]['DATE']

In [53]:
df_train = df.loc[df['DATE'] <= date]
df_test = df.loc[df['DATE'] > date]

**Exclude seasonal for training**

In [ ]:
# df_train = df_train[~df_train['MATERIAL'].isin(seasonal_list)]
# df_test = df_test[~df_test['MATERIAL'].isin(seasonal_list)]

In [54]:
print(df_train.shape)
print(df_test.shape)

(279417, 18)
(3954, 18)


**Encode**

In [55]:
cust_dummies_train = pd.get_dummies(df_train['CUSTNAME'])
cust_dummies_test = pd.get_dummies(df_test['CUSTNAME'])

In [56]:
df_train = pd.concat([df_train, cust_dummies_train], axis=1)
df_test = pd.concat([df_test, cust_dummies_test], axis=1)

In [57]:
df_train_x = df_train[['DATE', 'MATERIAL', 'CUSTNAME', 'Store 1', 'Store 2', 'Store 3', 'Store 4',
                       'Store 5', 'Store 6',
                       'QTY_SOLD_LY',
                       'QTY_SOLD_LM', 'QTY_SOLD_LW', 'Phoon Huat Season']]
df_train_y = df_train[['DATE', 'MATERIAL', 'CUSTNAME', 'QTY_SOLD']]
df_test_x = df_test[['DATE', 'MATERIAL', 'CUSTNAME', 'Store 1', 'Store 2', 'Store 3', 'Store 4',
                     'Store 5', 'Store 6', 'QTY_SOLD_LY',
                     'QTY_SOLD_LM', 'QTY_SOLD_LW', 'Phoon Huat Season']]
df_test_y = df_test[['DATE', 'MATERIAL', 'CUSTNAME', 'QTY_SOLD']]

In [58]:
df_train_x = df_train_x.set_index(['DATE', 'MATERIAL', 'CUSTNAME'])
df_test_x = df_test_x.set_index(['DATE', 'MATERIAL', 'CUSTNAME'])
df_train_y = df_train_y.set_index(['DATE', 'MATERIAL', 'CUSTNAME'])
df_test_y = df_test_y.set_index(['DATE', 'MATERIAL', 'CUSTNAME'])

In [59]:
lm = linear_model.LinearRegression()
model = lm.fit(df_train_x, df_train_y)
predictions = lm.predict(df_test_x)

In [60]:
len(predictions)

3954

In [61]:
predictions

array([[3.25195312],
       [3.25195312],
       [5.02043152],
       ...,
       [5.74760437],
       [9.01261902],
       [3.5552063 ]])

In [62]:
df_test_y['PREDICTED_QTY_SOLD'] = predictions

In [63]:
lm.score(df_train_x, df_train_y)

0.7407331893580094

In [64]:
np.sqrt(mean_squared_error(
    df_test_y['QTY_SOLD'], df_test_y['PREDICTED_QTY_SOLD']))

10.622400949462333

In [ ]:
df_train_x.head()

In [ ]:
df_train_x['Phoon Huat Season'].unique()

QTY_SOLD  PREDICTED_QTY_SOLD
DATE       MATERIAL CUSTNAME                              
2021-03-29 100498   Store 4        2.0            3.251953
           10147    Store 4        5.0            3.251953
           102139   Store 4        5.0            5.020432
           102574   Store 4        1.0            7.647018
           102619   Store 4        2.0            4.578644
...                                ...                 ...
           934      Store 3        8.0            6.513733
           9390     Store 3        1.0            2.752045
           9561     Store 3        5.0            5.747604
           9863     Store 3        5.0            9.012619
           9889     Store 3        4.0            3.555206

[3954 rows x 2 columns]

In [65]:
lm.coef_

array([[ 7.63828129e+10,  7.63828129e+10,  7.63828129e+10,
         7.63828129e+10,  7.63828129e+10,  7.63828129e+10,
         1.44287109e-01,  1.35353088e-01,  6.67812347e-01,
        -5.60774251e+00]])